# The lessived
## Trailer

In [1]:
%%HTML
<iframe width="560" height="315" src="./trailer.mp4" title="trailer" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>


## Jouer

Pour jouer, il suffit d'exécuter la cellule suivante :

Si vous lancez le jeu pour la première fois, l'exécution peut prendre quelques secondes. 

In [ ]:
%pip install pygame
import menu
menu.menu()

## Le jeu

Ce jeu est un jeu de type shooter à Zombies en mode vue du dessus. Il a été developpé par Térence DE SAINT LÉGER, Paul JÉRÔME--FILIO et Anatole BLANDIN.

### Les contôles
- Z, Q, S et D ou ↑, ←, ↓ et → pour se déplacer.
- A pour ouvrir l'inventaire (cette action met automatiquement le jeu en pause).
- Dans l'inventaire, LMB pour sélectionner un objet et RMB pour le déplacer
- E pour interragir avec une boîte.

### Les fonctionnalités
Ce jeu est un open world zombie shooter.

Actuellement, il y a :
- 6 armes uniques
- 5 types de zombies
- 3 powers ups différents
- 17 objets obtenables

### Détails techniques
#### La trajectoire des zombies
La trajectoire des zombies a été un véritable-casse tête de pensée. Au début, nous avions simplement tenté de faire se rapprocher les zombies des axes centraux de l'écran, puisque le personnage principal du jeu se trouve au centre de l'écran. Mais le mouvement n'était pas fluide : les zombies se déplaçaient vers les axes et non pas directement vers le centre.
Nous avons donc tenté de les faire se déplacer vers le centre de façon plus réelle, car l'expérience de jeu du joueur est une chose que nous tenons à cœur. Sur ce point nous avons eu pas mal de défaites (par exemples les zombies qui s'éloignaient en spirale du centre de l'écran), mais avons finalement réussi en utilisant les coordonnées du vecteur unitaire des zombies vers le centre.

De façon plus concrète :

Soient $A({x}_{A},{y}_{A})$ et $B({x}_{B},{y}_{B})$ deux points du plan. $B$ est un zombie et $A$ le personnage principal. Le vecteur unitaire $\overrightarrow{v}$ de direction $(AB)$ et de sens $B$ vers $A$ est donc :
\begin{align*}
    \overrightarrow{v}&=\frac{\overrightarrow{BA}}{\left|\left|\overrightarrow{BA}\right|\right|}\\
    &\\
    \overrightarrow{v}&=\frac{\overrightarrow{\binom{{x}_{A}-{x}_{B}}{{y}_{A}-{y}_{B}}}}{\left|\left|\overrightarrow{\binom{{x}_{A}-{x}_{B}}{{y}_{A}-{y}_{B}}}\right|\right|}\\
    &\\
    \overrightarrow{v}&=\frac{\overrightarrow{\binom{{x}_{A}-{x}_{B}}{{y}_{A}-{y}_{B}}}}{\sqrt{{\left({x}_{A}-{x}_{B}\right)}^{2}+{\left({y}_{A}-{y}_{B}\right)}^{2}}}\\
    &\\
    \overrightarrow{v}&=\overrightarrow{\binom{\frac{{{x}_{A}-{x}_{B}}}{\sqrt{{\left({x}_{A}-{x}_{B}\right)}^{2}+{\left({y}_{A}-{y}_{B}\right)}^{2}}}}{\frac{{{y}_{A}-{y}_{B}}}{\sqrt{{\left({x}_{A}-{x}_{B}\right)}^{2}+{\left({y}_{A}-{y}_{B}\right)}^{2}}}}}
\end{align*}

Ceci est implémenté dans ce petit bout de code :

```python
un_sur_l = Q_rsqrt((self.x - x/2)**2 + (self.y - y/2)**2)
self.vect = [un_sur_l * (self.x - x/2), un_sur_l * (self.y - y/2)]
```

#### L'orientaion du héros et des armes
Il fallait évidemment que le héros regarde la souris, et donc tourne sur lui-même, et il en va de même pour les armes. On pouvait facilement récupérer les coordonnées de la souris sur la fenêtre de jeu, mais encore fallait-il savoir les utiliser correctment.\
Bien sûr, if faut utiliser la trigonométrie ! Pour ça c'est plus ou moins facile en soi :\
Soit $x$ la largeur de la fenêtre de jeu, et soit $y$ sa hauteur. Notons $M\left({x}_{M},{y}_{M}\right)$ le poit représentant la souris. Alors l'angle $\alpha$ qui indique de quel angle il faut tourner le héros est de :\
$$\alpha = \arctan\left(\frac{{y}_{M}-\frac{y}{2}}{{x}_{M}-\frac{x}{2}}\right)$$
Purement mathématiquement, c'est assez facile, mais en terme d'implémentation, c'est plus dûr ! En effet, le module ```math``` de Python travaille avec des angles en radians, tandis que le module ```pygame```, notre moteur graphique, travaille en degrés. Donc voici le code :
```python
if mousepos[0]-x/2 != 0 :
    self.angle = math.atan((mousepos[1]-y/2)/(mousepos[0]-x/2))
    self.angle = convert_degrees(self.angle) # Pour pygame
    if mousepos[0] < x/2 : # Si la souris se situe dans la partie supérieure de l'écran
        self.angle = 180-self.angle
    else : # Sinon
        self.angle = -self.angle
```
Mais il se trouve que $\arctan$ ne renvoie que des nombres positifs entre $0$ et $1$ ! Donc on ne sait pas si la souris est en haut ou en bas sur l'écran, il faut faire des tests en plus pour connaître l'angle réel avec lequel tourner le héros.

#### La trajectoire des munitions
La trajectoire des munitions, c'est la notion physique et mathématique sans doute la plus complexe de notre jeu, mais aussi un des meilleurs rendus du jeu.\
Déjà, il faut que la munition parte de l'arme, et donc doit être décalée du centre en prenant en compte l'orientation de l'arme (c'est déjà tellement compliqué qu'on ne va pas détailler ce point). Ensuite, on garde l'angle de l'arme auquel on ajoute l'angle de dispersion de l'arme, et à partir de ces informations, on en déduit le vecteur unitaire directeur du mouvement plus ou moins de la même façon que pour la trajectoire des zombies. Bref, un enfer !

#### La génération de l'herbe
L'herbe est belle et réaliste dans notre jeu… bon, on s'emporte peut-être un peu ! Mais c'est vrai, on a un terrain infini, un open-world ! Mais comment faire pour générer l'herbe à l'infini sans avoir une image de taille infinie ou un nombre infini de variable ? On pourrait stocker les coordonnées des lieux visités par le héros dans une liste et afficher de l'herbe à chacune de ces coordonnées, mais plus on voyagerait dans notre open-world et plus la liste serait longue, et votre ordinateur ne serait pas très content d'avoir à gérer tout ça. Donc crée 9 images, chacune correspondant à un pavage du plan autour du personnage pricipal, en suivant les point cardinaux (```H``` représente le héros) :

```
NO    N    N
   \  |  /
O --  H  -- E
   /  |  \
SO    S    SE
```

## Jouer
N'oubliez pas de jouer, en exécutant la cellule suivante :

In [ ]:
%pip install pygame
import menu
menu.menu()